In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

In [ ]:
a = torch.randn(4, 4)
print(a,a.shape)
torch.min(a, 0)

tensor([[-0.0260,  2.2843, -1.2085, -0.1291],
        [-1.7867,  0.9784,  0.9293, -0.8415],
        [ 1.0278,  0.5765,  0.5358, -0.5194],
        [ 0.4338, -0.8639,  0.4175, -0.5844]]) torch.Size([4, 4])


torch.return_types.min(
values=tensor([-1.7867, -0.8639, -1.2085, -0.8415]),
indices=tensor([1, 3, 0, 1]))

In [ ]:
# Check that MPS is available

device='mps' if torch.backends.mps.is_available() else 0


if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
            "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")

    # Create a Tensor directly on the mps device
    x = torch.ones(5, device=mps_device)
    # Or
    x = torch.ones(5, device="mps")
    
    print(x)

    # Any operation happens on the GPU
    y = x * 2
    


tensor([1., 1., 1., 1., 1.], device='mps:0')


In [3]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModel

model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')


In [2]:
from transformers import AutoTokenizer, AutoModel

# Sentences we want sentence embeddings for
sentence1 = 'This is an example sentence hashin"s'
sentence2='he sarthak i ma so nice to meet you sonn and how have you been '

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model =('sentence-transformers/all-MiniLM-L12-v2')

# Tokenize sentences
encoded_input1 = tokenizer(sentence1, padding='longest', truncation=True, 
                        return_tensors='pt')

In [17]:
encoded_input1['input_ids'].squeeze(0)

tensor([  101,  2023,  2003,  2019,  2742,  6251, 23325,  2378,  1000,  1055,
          102])

In [18]:
[1]*11

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [15]:
(encoded_input1['attention_mask'].squeeze(0)*len(encoded_input1['input_ids'].squeeze(0)))

tensor([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11])

In [32]:
encoded_input1['attention_mask'].unsqueeze(-1).expand(1, 11, 384)

RuntimeError: The expanded size of the tensor (12) must match the existing size (11) at non-singleton dimension 1.  Target sizes: [1, 12, 384].  Tensor sizes: [1, 11, 1]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    
    #print(model_output.last_hidden_state.shape)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
#sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings.shape)


torch.Size([2, 7, 384])
Sentence embeddings:
torch.Size([2, 384])


In [37]:
output=model(**encoded_input1)

In [42]:
output.last_hidden_state==model(**encoded_input1)[0]

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [8]:
from config import *

for name,param in model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [46]:
encoded_input1['input_ids'].shape

torch.Size([1, 11])

In [53]:
encoded_input1['input_ids'][0].shape

torch.Size([11])

In [47]:
encoded_input2 = tokenizer(sentence2, padding='longest', truncation=True, 
                        return_tensors='pt')

In [52]:
encoded_input2['input_ids'].size()

torch.Size([1, 20])

In [92]:
len(encoded_input1['input_ids'][0])

7

In [17]:
len(encoded_input1['input_ids'][0])==len(encoded_input1['token_type_ids'][0])

True

In [94]:
pad_len=max(len(encoded_input1['input_ids'][0]),len(encoded_input2['input_ids'][0]))
pad_len

7

In [54]:
joint=pad_sequence([encoded_input1['input_ids'][0],encoded_input2['input_ids'][0]],batch_first=True).unsqueeze(1)

joint1=pad_sequence([encoded_input1['token_type_ids'][0],encoded_input2['token_type_ids'][0]],batch_first=True).unsqueeze(1)

joint2=pad_sequence([encoded_input1['attention_mask'][0],encoded_input2['attention_mask'][0]],batch_first=True).unsqueeze(1)



In [8]:
model(joint[1],joint1[1],joint2[1])

NameError: name 'joint' is not defined

In [40]:
encoded_input=torch.cat((encoded_input1['input_ids'],encoded_input2['input_ids']),-1)

encoded_input.shape

torch.Size([1, 160])

In [33]:
encoded_input2['input_ids'].shape

torch.Size([1, 80])

In [63]:
input_ids=encoded_input['input_ids']
attn_ids=encoded_input['attention_mask']
tkn_ids=encoded_input['token_type_ids']

In [7]:
model_output = model(input_ids,attn_ids,tkn_ids,output_hidden_states=True)


NameError: name 'input_ids' is not defined

In [102]:
model_output.last_hidden_state[:,0,:].shape

torch.Size([3, 384])

In [66]:
model_output[0].mean(dim=1).shape

torch.Size([1, 384])

In [67]:
model_output.last_hidden_state.shape

torch.Size([1, 7, 384])

In [83]:
encoded_input['attention_mask'].unsqueeze(-1).shape

torch.Size([3, 7, 1])

In [80]:
encoded_input['attention_mask'].unsqueeze(-1).expand(model_output[0].shape)

tensor([[[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 0]]])

In [86]:
token_embeddings = model_output[0] #First element of model_output contains all token embeddings
input_mask_expanded = attn_ids.unsqueeze(-1).expand(token_embeddings.size()).float()
torch.sum(token_embeddings * input_mask_expanded,1)/

tensor([[-0.0062,  2.4783,  0.9539,  ...,  1.8886,  1.4935, -0.0088],
        [ 1.2822,  1.0358, -3.5860,  ..., -1.8445, -0.9747,  1.1293],
        [-2.2973,  0.0492, -1.5071,  ...,  0.7316, -0.2213, -1.1986]],
       grad_fn=<SumBackward1>)

In [138]:
model_output[0].mean(1).shape

torch.Size([3, 384])

In [92]:
torch.sum(token_embeddings * input_mask_expanded,1)/torch.clamp(torch.sum(input_mask_expanded,1),min=1e-9)

tensor([[-0.0009,  0.3540,  0.1363,  ...,  0.2698,  0.2134, -0.0013],
        [ 0.1832,  0.1480, -0.5123,  ..., -0.2635, -0.1392,  0.1613],
        [-0.3829,  0.0082, -0.2512,  ...,  0.1219, -0.0369, -0.1998]],
       grad_fn=<DivBackward0>)

In [129]:
torch.rand(20,256,384).view(20,-1).shape

torch.Size([20, 98304])

In [141]:
tokenizer.model_max_length

512

In [139]:
model.config

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "sentence-transformers/all-MiniLM-L12-v2",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [54]:
torch.tensor((1,2),dtype=float)

tensor([1., 2.], dtype=torch.float64)

In [65]:
import torch 

# Initializing a linear layer with 
# 2 independent features and 3 dependent features 
linear_layer = torch.nn.Parameter(torch.rand((2,4),dtype=float)) 

# Initializing the weights with the Xavier initialization method 
torch.nn.init.xavier_uniform_(linear_layer) 

# Displaying the initialized weights 
print(linear_layer) 


Parameter containing:
tensor([[-0.0358,  0.0416,  0.6203, -0.7728],
        [-0.8243, -0.9608,  0.3213, -0.9329]], dtype=torch.float64,
       requires_grad=True)


## Dataset

In [20]:
import pandas as pd

df=pd.read_csv('../data/llm_train.csv')
df['question'][1212]

'what is a home remedy for gallstone cleansing?'

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(["amsterdam",'tokyo','amsterdam','paris'])
list(le.classes_)
le.transform(["tokyo", "amsterdam",'paris'])
#list(le.inverse_transform([2, 2, 1]))

array([2, 0, 1])

In [18]:
le = LabelEncoder()
le.fit(['paris',"tokyo", "amsterdam"])
list(le.classes_)
le.transform(['paris',"tokyo", "tokyo", "amsterdam"])
#list(le.inverse_transform([2, 2, 1]))

array([1, 2, 2, 0])

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)  # Use torch.long for classification labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Example data
features = [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]]
labels = [0, 1, 0]

# Create dataset
dataset = MyDataset(features, labels)

# Create data loader for batch processing
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

for data, target in data_loader:
    print('Features:', data, 'Labels:', target)


Features: tensor([[0.1000, 0.2000],
        [0.3000, 0.4000]]) Labels: tensor([0, 1])
Features: tensor([[0.5000, 0.6000]]) Labels: tensor([0])


In [1]:
import torch.nn as nn 


In [3]:
# Example of target with class indices
import torch.nn as nn 

loss = nn.CrossEntropyLoss()

In [121]:

input = torch.tensor([1],dtype=float,requires_grad=True)
target = torch.randn(1).softmax(-1)
output = loss(input, target)
output.backward()


# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [122]:
input

tensor([[ 0.9645, -2.0319, -0.6062,  0.0788,  0.0551],
        [-1.4662,  0.8733,  0.9075,  1.0510, -1.7239],
        [ 0.8049,  1.8266,  0.4291, -0.1938,  0.3185]], requires_grad=True)

In [119]:
target

tensor([4, 2, 3])

In [115]:

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

In [116]:
target

tensor([[0.0681, 0.3211, 0.1859, 0.3497, 0.0751],
        [0.1168, 0.0960, 0.6239, 0.1229, 0.0404],
        [0.1095, 0.2218, 0.2046, 0.1339, 0.3302]])

In [61]:
target

tensor([[0.0507, 0.4579, 0.0463, 0.2631, 0.1820],
        [0.0992, 0.0373, 0.6464, 0.1523, 0.0648],
        [0.2710, 0.1301, 0.0457, 0.2898, 0.2635]])

In [112]:
arr=torch.rand(2,5).softmax(-1)
torch.argmax(arr,-1)

tensor([1, 3])

In [1]:
from torch.nn.utils.rnn import pad_sequence
a = torch.ones(25, 300)
b = torch.ones(25, 300)
c = torch.ones(25, 30)
pad_sequence([a, b, c],batch_first=True).size()

NameError: name 'torch' is not defined

In [9]:
protos=torch.nn.Parameter(torch.rand(2,5))

torch.dot(protos[0],protos[1]).requires_grad

True

In [28]:
import torch
import torch.nn as nn

# Example with a batch size of 3 and 5 classes
batch_size = 3
num_classes = 5
input_tensor = torch.randn(batch_size, num_classes) # Model output (logits)
labels = torch.randint(0, num_classes, (batch_size,)) # Ground truth labels
print(input_tensor,labels)
criterion = nn.CrossEntropyLoss()
loss = criterion(input_tensor, labels)

print(f"Input shape: {input_tensor.shape}")
print(f"Labels shape: {labels.shape}")


soft=nn.Softmax(dim=-1)
print(f"Softmax :{soft(input_tensor)}")
print(f"Loss: {loss.item()}")

tensor([[ 0.2326, -1.8181,  1.0245, -0.1856, -2.0433],
        [-1.0472, -0.0765,  0.4456, -0.8408,  0.9999],
        [ 0.6249,  1.5793, -0.3619,  0.5813, -0.9429]]) tensor([1, 4, 3])
Input shape: torch.Size([3, 5])
Labels shape: torch.Size([3])
Softmax :tensor([[0.2441, 0.0314, 0.5388, 0.1607, 0.0251],
        [0.0586, 0.1547, 0.2608, 0.0720, 0.4539],
        [0.1947, 0.5057, 0.0726, 0.1864, 0.0406]])
Loss: 1.9768867492675781
